In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set a style for plots
sns.set_style("whitegrid")

# Load the Benin dataset
# IMPORTANT: Double-check this path. If your 'data' folder is directly in 'solar-challenge-week1/',
# and your 'notebooks' folder is also directly in 'solar-challenge-week1/', then this path is correct.
file_path_benin = '../data/benin-malanville.csv'

try:
    df_benin = pd.read_csv(file_path_benin)
except FileNotFoundError:
    print(f"Error: {file_path_benin} not found. Please ensure the 'benin-malanville.csv' file is in a 'data' folder next to your 'notebooks' folder.")
    # Re-raise the error so that subsequent cells don't try to run without data
    raise

# Convert 'Timestamp' to datetime
df_benin['Timestamp'] = pd.to_datetime(df_benin['Timestamp'])
df_benin = df_benin.sort_values(by='Timestamp')

print("DataFrame head:")
print(df_benin.head())
print("\nDataFrame info:")
df_benin.info()

DataFrame head:
            Timestamp  GHI  DNI  DHI  ModA  ModB  Tamb    RH   WS  WSgust  \
0 2021-08-09 00:01:00 -1.2 -0.2 -1.1   0.0   0.0  26.2  93.4  0.0     0.4   
1 2021-08-09 00:02:00 -1.1 -0.2 -1.1   0.0   0.0  26.2  93.6  0.0     0.0   
2 2021-08-09 00:03:00 -1.1 -0.2 -1.1   0.0   0.0  26.2  93.7  0.3     1.1   
3 2021-08-09 00:04:00 -1.1 -0.1 -1.0   0.0   0.0  26.2  93.3  0.2     0.7   
4 2021-08-09 00:05:00 -1.0 -0.1 -1.0   0.0   0.0  26.2  93.3  0.1     0.7   

   WSstdev     WD  WDstdev   BP  Cleaning  Precipitation  TModA  TModB  \
0      0.1  122.1      0.0  998         0            0.0   26.3   26.2   
1      0.0    0.0      0.0  998         0            0.0   26.3   26.2   
2      0.5  124.6      1.5  997         0            0.0   26.4   26.2   
3      0.4  120.3      1.3  997         0            0.0   26.4   26.3   
4      0.3  113.2      1.0  997         0            0.0   26.4   26.3   

   Comments  
0       NaN  
1       NaN  
2       NaN  
3       NaN  
4     

In [2]:
# Summary Statistics
print("\nSummary Statistics for Numeric Columns:")
print(df_benin.describe())

# Missing Values Report
print("\nMissing Values Count:")
missing_values = df_benin.isnull().sum()
print(missing_values[missing_values > 0])

print("\nColumns with >5% Nulls:")
total_rows = len(df_benin)
for column, count in missing_values.items():
    if count / total_rows > 0.05:
        print(f"- {column}: {count} ({count/total_rows:.2%})")

# We already know 'Comments' is empty, let's confirm
if 'Comments' in df_benin.columns and df_benin['Comments'].isnull().all():
    print("\n'Comments' column is entirely null and will be considered for removal.")


Summary Statistics for Numeric Columns:
                           Timestamp            GHI            DNI  \
count                         525600  525600.000000  525600.000000   
mean   2022-02-07 12:00:30.000000512     240.559452     167.187516   
min              2021-08-09 00:01:00     -12.900000      -7.800000   
25%              2021-11-08 06:00:45      -2.000000      -0.500000   
50%              2022-02-07 12:00:30       1.800000      -0.100000   
75%              2022-05-09 18:00:15     483.400000     314.200000   
max              2022-08-09 00:00:00    1413.000000     952.300000   
std                              NaN     331.131327     261.710501   

                 DHI           ModA           ModB           Tamb  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      115.358961     236.589496     228.883576      28.179683   
min       -12.600000       0.000000       0.000000      11.000000   
25%        -2.100000       0.000000       0.000000  

In [3]:
# --- Outlier Detection & Basic Cleaning ---
print("\n--- Outlier Detection & Basic Cleaning ---")

# Columns to check for outliers using Z-scores
outlier_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Calculate Z-scores and flag rows with |Z|>3
for col in outlier_cols:
    if col in df_benin.columns:
        df_benin[f'Zscore_{col}'] = np.abs((df_benin[col] - df_benin[col].mean()) / df_benin[col].std())
        outliers = df_benin[df_benin[f'Zscore_{col}'] > 3]
        print(f"\nPotential outliers in '{col}' (|Z|>3): {len(outliers)} rows")
        if not outliers.empty:
            print(outliers[[col, 'Timestamp', f'Zscore_{col}']].head())
    else:
        print(f"Warning: Column '{col}' not found for outlier detection.")

# Basic Cleaning: Dropping the 'Comments' column as it's entirely null
if 'Comments' in df_benin.columns and df_benin['Comments'].isnull().all():
    df_benin = df_benin.drop(columns=['Comments'])
    print("\n'Comments' column dropped as it contained only null values.")

# Handling other missing values: instructions say 'Drop or impute (median) missing values in key columns.'
# Based on our initial check, there were no missing values in key columns like GHI, DNI etc.
# If any were found, we would add imputation or dropping logic here.
print("\nMissing values in key columns (after initial check, should be zero or very few):")
print(df_benin[outlier_cols].isnull().sum())

# Export cleaned DataFrame (important for Task 3)
# Create 'data' directory if it doesn't exist
import os
output_dir = '../data' # This path is relative to the notebooks folder
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

output_path_benin_clean = os.path.join(output_dir, 'benin_clean.csv')
df_benin.to_csv(output_path_benin_clean, index=False)
print(f"\nCleaned Benin data exported to: {output_path_benin_clean}")


--- Outlier Detection & Basic Cleaning ---

Potential outliers in 'GHI' (|Z|>3): 89 rows
        GHI           Timestamp  Zscore_GHI
671  1274.0 2021-08-09 11:12:00    3.120939
674  1349.0 2021-08-09 11:15:00    3.347435
676  1334.0 2021-08-09 11:17:00    3.302136
849  1253.0 2021-08-09 14:10:00    3.057520
850  1324.0 2021-08-09 14:11:00    3.271936

Potential outliers in 'DNI' (|Z|>3): 0 rows

Potential outliers in 'DHI' (|Z|>3): 3738 rows
       DHI           Timestamp  Zscore_DHI
670  610.6 2021-08-09 11:11:00    3.120787
671  615.2 2021-08-09 11:12:00    3.149774
672  612.8 2021-08-09 11:13:00    3.134650
673  593.2 2021-08-09 11:14:00    3.011140
674  618.0 2021-08-09 11:15:00    3.167418

Potential outliers in 'ModA' (|Z|>3): 27 rows
        ModA           Timestamp  Zscore_ModA
674   1281.5 2021-08-09 11:15:00     3.196473
676   1267.3 2021-08-09 11:17:00     3.153034
850   1218.0 2021-08-09 14:11:00     3.002221
5019  1271.0 2021-08-12 11:40:00     3.164352
5024  1305.6 2021-

In [4]:
# --- Outlier Detection & Basic Cleaning ---
print("\n--- Outlier Detection & Basic Cleaning ---")

# Columns to check for outliers using Z-scores
outlier_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Calculate Z-scores and flag rows with |Z|>3
for col in outlier_cols:
    if col in df_benin.columns:
        df_benin[f'Zscore_{col}'] = np.abs((df_benin[col] - df_benin[col].mean()) / df_benin[col].std())
        outliers = df_benin[df_benin[f'Zscore_{col}'] > 3]
        print(f"\nPotential outliers in '{col}' (|Z|>3): {len(outliers)} rows")
        if not outliers.empty:
            print(outliers[[col, 'Timestamp', f'Zscore_{col}']].head())
    else:
        print(f"Warning: Column '{col}' not found for outlier detection.")

# Basic Cleaning: Dropping the 'Comments' column as it's entirely null
if 'Comments' in df_benin.columns and df_benin['Comments'].isnull().all():
    df_benin = df_benin.drop(columns=['Comments'])
    print("\n'Comments' column dropped as it contained only null values.")

# Handling other missing values: instructions say 'Drop or impute (median) missing values in key columns.'
# Based on our initial check, there were no missing values in key columns like GHI, DNI etc.
# If any were found, we would add imputation or dropping logic here.
print("\nMissing values in key columns (after initial check, should be zero or very few):")
print(df_benin[outlier_cols].isnull().sum())

# Export cleaned DataFrame (important for Task 3)
# Create 'data' directory if it doesn't exist
import os
output_dir = '../data' # This path is relative to the notebooks folder
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"Created directory: {output_dir}")

output_path_benin_clean = os.path.join(output_dir, 'benin_clean.csv')
df_benin.to_csv(output_path_benin_clean, index=False)
print(f"\nCleaned Benin data exported to: {output_path_benin_clean}")


--- Outlier Detection & Basic Cleaning ---

Potential outliers in 'GHI' (|Z|>3): 89 rows
        GHI           Timestamp  Zscore_GHI
671  1274.0 2021-08-09 11:12:00    3.120939
674  1349.0 2021-08-09 11:15:00    3.347435
676  1334.0 2021-08-09 11:17:00    3.302136
849  1253.0 2021-08-09 14:10:00    3.057520
850  1324.0 2021-08-09 14:11:00    3.271936

Potential outliers in 'DNI' (|Z|>3): 0 rows

Potential outliers in 'DHI' (|Z|>3): 3738 rows
       DHI           Timestamp  Zscore_DHI
670  610.6 2021-08-09 11:11:00    3.120787
671  615.2 2021-08-09 11:12:00    3.149774
672  612.8 2021-08-09 11:13:00    3.134650
673  593.2 2021-08-09 11:14:00    3.011140
674  618.0 2021-08-09 11:15:00    3.167418

Potential outliers in 'ModA' (|Z|>3): 27 rows
        ModA           Timestamp  Zscore_ModA
674   1281.5 2021-08-09 11:15:00     3.196473
676   1267.3 2021-08-09 11:17:00     3.153034
850   1218.0 2021-08-09 14:11:00     3.002221
5019  1271.0 2021-08-12 11:40:00     3.164352
5024  1305.6 2021-

In [5]:
# Summary Statistics
print("\nSummary Statistics for Numeric Columns:")
print(df_benin.describe())

# Missing Values Report
print("\nMissing Values Count:")
missing_values = df_benin.isnull().sum()
print(missing_values[missing_values > 0])

print("\nColumns with >5% Nulls:")
total_rows = len(df_benin)
for column, count in missing_values.items():
    if count / total_rows > 0.05:
        print(f"- {column}: {count} ({count/total_rows:.2%})")

# We already know 'Comments' is empty, let's confirm
if 'Comments' in df_benin.columns and df_benin['Comments'].isnull().all():
    print("\n'Comments' column is entirely null and will be considered for removal.")


Summary Statistics for Numeric Columns:
                           Timestamp            GHI            DNI  \
count                         525600  525600.000000  525600.000000   
mean   2022-02-07 12:00:30.000000512     240.559452     167.187516   
min              2021-08-09 00:01:00     -12.900000      -7.800000   
25%              2021-11-08 06:00:45      -2.000000      -0.500000   
50%              2022-02-07 12:00:30       1.800000      -0.100000   
75%              2022-05-09 18:00:15     483.400000     314.200000   
max              2022-08-09 00:00:00    1413.000000     952.300000   
std                              NaN     331.131327     261.710501   

                 DHI           ModA           ModB           Tamb  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      115.358961     236.589496     228.883576      28.179683   
min       -12.600000       0.000000       0.000000      11.000000   
25%        -2.100000       0.000000       0.000000  